# Problem sugestion with MAB algorithms

We are using a simple MAB algorithm to select problems to competitive programming competitors. In this work we first introduce the base case.

First we start with the Epsilon-Greedy algorithm.

## The data

The algorithm will be trainning using the data from the URI Onlide Judge. We solve around of 200k solutions.
The solutions is a list of tuples composed by `(problem_id, user_id, date)`.

At first we will read the data and sort by date.

In [214]:
import random, math
from itertools import groupby
from collections import namedtuple
import pandas as pd
import matplotlib.pyplot as plt

In [220]:
solutions_df = pd.read_csv('fake_solutions.csv')
solutions_df[:5]

,user,problem,category,date
0,1,1499,1,0
1,1,1498,1,1
2,1,1497,1,2
3,1,1496,1,3
4,1,1495,1,4


The first thing to do after read the dataset, is filter the data. This is important for us, because doint this first, we need to work with less data. This will speed up our work.

In [221]:
solutions_df = solutions_df.loc[solutions_df['category'] == 1]
solutions_df[:5]

,user,problem,category,date
0,1,1499,1,0
1,1,1498,1,1
2,1,1497,1,2
3,1,1496,1,3
4,1,1495,1,4


Now, we will get the set of problems and users.

In [222]:
problems = list(set(solutions_df['problem']))
problems.sort()
problems[:5]

[1002, 1003, 1004, 1005, 1006]

In [223]:
users = list(set(solutions_df['user']))
users.sort()
users[:5]

[1, 2, 3, 4, 5]

Now we will fix the user and problems id. We will make they be continuous starting from 0.

In [224]:
for i, user in enumerate(users):
    solutions_df.replace({'user': {user: i}}, inplace=True)

for i, problem in enumerate(problems):
    solutions_df.replace({'problem': {problem: i}}, inplace=True)
    
solutions_df[:5]

,user,problem,category,date
0,0,497,1,0
1,0,496,1,1
2,0,495,1,2
3,0,494,1,3
4,0,493,1,4


Update the users and problems to correspond the new id's

In [225]:
problems = list(set(solutions_df['problem']))
problems[:5]

[0, 1, 2, 3, 4]

In [226]:
users = list(set(solutions_df['user']))
users[:5]

[0, 1, 2, 3, 4]

After that, we need to sort the data by the date.

In [227]:
solutions_df = solutions_df.sort_values('date')
solutions_df[:5]

,user,problem,category,date
0,0,497,1,0
388842,1568,497,1,0
388555,1567,497,1,0
59793,245,497,1,0
59904,246,497,1,0


Now, we can split the data. 
OBS: The problems are indexed py your id. How they are sequential, we will decrease the value 1001 of each one. Because this will make the code more simple.

In [228]:
Solution = namedtuple('Solution', 'problem, user')
solutions = []

for row in solutions_df.itertuples():
    solutions.append(
        Solution(row.problem, row.user)
    )
        
solutions[:5]

[Solution(problem=497, user=0),
 Solution(problem=497, user=1568),
 Solution(problem=497, user=1567),
 Solution(problem=497, user=245),
 Solution(problem=497, user=246)]

At this point we have everything that is necessary to test our algorithms. To start, we will use the Epsilon-Greedy algorithm

In [229]:
class EpsilonGreedy():
    def __init__(self, n_users, n_problems):
        
        self.epsilon = 0.1
        self.n_arms = n_problems
        self.counts = [0     for col in range(self.n_arms)]
        self.values = [0.0   for col in range(self.n_arms)]
        self.chosen = [[0 for col in range(n_problems)] for col in range(n_users)]


    def max_arm(self, user):
        max_arm = 0
        max_value = -1
        
        for arm in range(self.n_arms):
            if self.chosen[user][arm]:
                continue
                
            if self.values[arm] > max_value:
                max_arm = arm
                max_value = self.values[arm]

        return max_arm

    
    def rand_arm(self, user):
        rand_arm = random.randrange(self.n_arms)
        
        while self.chosen[user][rand_arm]:
            rand_arm = (rand_arm + 1) % self.n_arms

        return rand_arm


    def select_arm(self, user):
        t = sum(self.counts) + 1
        epsilon = 1 / math.log(t + 0.0000001)
        
        if random.random() > epsilon:
            return self.max_arm(user)
        else:
            return self.rand_arm(user)
  

    def update(self, chosen_arm, reward):
        self.counts[chosen_arm] = self.counts[chosen_arm] + 1

        n = self.counts[chosen_arm]
        value = self.values[chosen_arm]
        new_value = ((n - 1) / float(n)) * value + (1 / float(n)) * reward
        self.values[chosen_arm] = new_value

And we have a test structure

In [230]:
def test_algorithm(algo, solutions):
    n_solutions        = len(solutions)
    rewards            = [0 for i in range(n_solutions)]
    cumulative_rewards = [0 for i in range(n_solutions)]
    chosen_arms        = [0 for i in range(n_solutions)]
  
    for i, solution in enumerate(solutions):
        chosen_arm = algo.select_arm(solution.user)

        if chosen_arm == solution.problem:
            reward = 1
        else:
            reward = 0
        
        rewards[i] = reward
        cumulative_rewards[i] = cumulative_rewards[i - 1] + reward
        chosen_arms[i] = chosen_arm
        
        algo.update(chosen_arm, reward)
        algo.chosen[solution.user][solution.problem] = 1

    return cumulative_rewards

So, for each user in the dataset, we will use the algorithm. And then plot the result of the accuracy of the algorithm.

In [ ]:
n_users = len(users)
n_problems = len(problems)

algo = EpsilonGreedy(n_users, n_problems)
cumulative_rewards = test_algorithm(algo, solutions)

In [ ]:
x = range(len(solutions))
y = cumulative_rewards
plt.scatter(x, y, alpha=0.5)
plt.title(u'Epsilon Greedy')
plt.xlabel('tempo')
plt.ylabel(u'recompensa acumulada')
plt.show()